In [1]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import pandas as pd

from transformers import pipeline
import os
import accelerate
import torch
from compliance_checker import *

/root/LLMX/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_name = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=5000,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.10it/s]
Device set to use cuda:0


In [2]:
belgium_guidelines = pd.read_csv("data/belgium_rules.csv")

# Initialize checker
# compliance_checker = NutritionComplianceChecker(belgium_guidelines, llm)

# Load recipe
recipes_df = pd.read_csv("data/data_kg.csv")
# sample_recipe = recipes_df.iloc[3].to_dict()
# report = compliance_checker.check_compliance(sample_recipe)

In [4]:
recipes_df.ingredients[3]

'18 ounces white cake mix, 3 eggs or as called for by your cake mix, 1/3 cup oil or as called for by your cake mix, 1 1/3 cups water or as called for by your cake mix, 1 15 ounce, can cream of coconut Coco Lopez, 1 8 ounce, container Cool Whip, 1 6 ounce, can flaked coconut'

In [6]:
recipes_df.columns

Index(['recipe_id', 'title', 'duration', 'ingredients', 'ingredient_list',
       'directions', 'allergens', 'meal_course', 'diet', 'dish_type'],
      dtype='object')

In [4]:
# Print results
print(f"\nCompliance Report for: {report['title']}")
print(f"Dish Type: {report['dish_type']}")
print(f"Meal Course: {report['meal_course']}")
print(f"Compliance Score: {report['compliance_score']}%")
print("\nGuideline Assessments:")
    
for result in report['results']:
    status_icon = "✅" if "Compliant" in result['status'] else "⚠️" if "Partially" in result['status'] else "❌"
    print(f"{status_icon} {result['guideline_id']}: {result['status']}")
    print(f"   Reason: {result['reason']}")
    print(f"   Guideline: {belgium_guidelines.iloc[int(result['guideline_id'][1:])-1]['rule']}")
    print()


Compliance Report for: Dreamy Coconut Cake
Dish Type: cake
Meal Course: ['Dessert']
Compliance Score: 23.08%

Guideline Assessments:
✅ G1: Compliant
   Reason: Uses whole wheat pasta instead of refined grains
   Guideline: Replace refined grains with whole grains, such as choosing wholegrain or wholemeal bread over white bread and wholemeal pasta over white pasta.

❌ G2: Context Mismatch
   Reason: Fish guideline not applicable to dessert recipes
   Guideline: Consume at least 125 g of whole-grain products daily.

✅ G3: Non-Compliant
   Reason: Contains 10g of added sugar exceeding 5g limit
   Guideline: A diet low in whole-grain products increases health risks, with a minimum health risk intake defined as 100 to 150 g per day.

✅ G69: Partially Compliant
   Reason: Uses dairy products, but does not specify the quantity consumed
   Guideline: Consume between 250 and 500 ml of milk or dairy products daily

✅ G83: Non-Compliant
   Reason: Contains tropical oil from coconut and palm tree

In [ ]:
df_guid=pd.read_json("data/belgian_organized_rule_list.jsonl", lines=True)

In [ ]:
df_guid

In [ ]:
[i for i in report['results'] if i['status']!="Not Applicable"]

In [ ]:
sample_recipe

In [ ]:
df_pp=pd.read_csv('data/pp_recipes.csv')